In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
random_seed = 42

In [2]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [3]:
X_train

,r1_x,r1_y,r2_x,r2_y,r3_x,r3_y,r4_x,r4_y,r5_x,r5_y,d1_x,d1_y,d2_x,d2_y,d3_x,d3_y,d4_x,d4_y,d5_x,d5_y
492,116.0,160.0,72.0,76.0,82.0,166.0,84.0,130.0,90.0,126.0,134.0,168.0,114.0,178.0,118.0,154.0,182.0,176.0,180.0,174.0
10529,84.0,168.0,170.0,82.0,130.0,130.0,90.0,164.0,86.0,166.0,92.0,172.0,90.0,170.0,170.0,92.0,174.0,86.0,130.0,130.0
366,114.0,110.0,98.0,116.0,72.0,78.0,92.0,118.0,104.0,122.0,92.0,140.0,100.0,120.0,102.0,154.0,158.0,94.0,140.0,122.0
3845,92.0,134.0,106.0,86.0,92.0,138.0,92.0,128.0,110.0,78.0,132.0,172.0,154.0,92.0,140.0,140.0,128.0,158.0,110.0,108.0
18722,178.0,102.0,168.0,96.0,84.0,134.0,74.0,90.0,102.0,114.0,76.0,156.0,108.0,114.0,124.0,124.0,152.0,130.0,160.0,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22255,74.0,78.0,86.0,82.0,80.0,82.0,78.0,78.0,78.0,86.0,96.0,78.0,116.0,108.0,84.0,80.0,180.0,174.0,184.0,174.0
5549,102.0,158.0,104.0,160.0,146.0,78.0,150.0,154.0,102.0,152.0,122.0,172.0,122.0,126.0,106.0,158.0,120.0,138.0,124.0,130.0
876,176.0,166.0,168.0,160.0,164.0,156.0,174.0,166.0,72.0,100.0,134.0,168.0,112.0,140.0,176.0,170.0,130.0,144.0,156.0,154.0
16284,92.0,138.0,78.0,130.0,122.0,94.0,78.0,130.0,116.0,118.0,148.0,122.0,116.0,118.0,132.0,130.0,148.0,128.0,128.0,126.0


In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
parameters = {
    'alpha' : [0.01, 0.1, 1, 2, 3, 5, 10, 50, 100],
}
#l = LinearRegression()
log = Ridge()
model = GridSearchCV(log, param_grid=parameters, cv= 5)
model.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0.1, 1, 2, 3, 5, 10, 50, 100]})

In [5]:
model.best_params_

{'alpha': 100}

In [6]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")


Score : 0.17216403160135296
ROC-AUC score : 0.7377105152665288


In [7]:
ut.write(model)

[0.37799978 0.3951319  0.55269517 ... 0.50679307 0.60683847 0.12760876]
